### Create additional validation data:

creates `0_pd_stats_mb_ablation_grad_ti_model_comparison.csv`
- here only the MB ablation gradient is estimated. The other, possibly validation data, i.e. mean absolute error to MB profile and interannual MB variability were created in `02_merging_calibration_options.ipynb`

In [98]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=20)
import warnings
warnings.filterwarnings("once", category=DeprecationWarning)  # noqa: E402
import scipy
# imports from OGGM
import oggm
from oggm import utils, workflow, tasks, cfg, entity_task
import numpy as np
import pandas as pd
from MBsandbox.mbmod_daily_oneflowline import (TIModel_Sfc_Type, process_w5e5_data)
from MBsandbox.wip.projections_bayescalibration import process_isimip_data, process_isimip_data_no_corr
from MBsandbox.help_func import (minimize_winter_mb_brentq_geod_via_pf, minimize_bias_geodetic,
                                 calibrate_to_geodetic_bias_winter_mb)
from MBsandbox.mbmod_daily_oneflowline import compile_fixed_geometry_mass_balance_TIModel
import time
import logging

log = logging.getLogger(__name__)

base_url = ('https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/'
            'L1-L2_files/elev_bands')
climate_type = 'W5E5'

# get the geodetic calibration data
pd_geodetic_all = utils.get_geodetic_mb_dataframe()
# pd_geodetic_all = pd.read_hdf(path_geodetic, index_col='rgiid')
pd_geodetic = pd_geodetic_all.loc[pd_geodetic_all.period == '2000-01-01_2020-01-01']


#pd_wgms_ref_glac_analysis = pd.read_csv('/home/lilianschuster/Schreibtisch/PhD/wgms_data_analysis/wgms_data_analysis.csv', index_col=[0])
#rgis_w_mb_profiles = pd_wgms_ref_glac_analysis[pd_wgms_ref_glac_analysis.MB_profile.dropna()].index

import MBsandbox
cluster = True


_, pathi = utils.get_wgms_files()
pd_mb_overview = pd.read_csv(pathi[:-len('/mbdata')] + '/mb_overview_seasonal_mb_time_periods_20220301.csv',
                             index_col='Unnamed: 0')
pd_wgms_data_stats = pd.read_csv(pathi[:-len('/mbdata')] + '/

# should have at least 5 annual MB estimates in the time period 1980-2019
# (otherwise can also not have MB profiles or winter MB!)
pd_wgms_data_stats = pd_wgms_data_stats.loc[pd_wgms_data_stats.len_annual_balance>=5]
ref_candidates = pd_wgms_data_stats.rgi_id.unique()


#_, path = utils.get_wgms_files()
#pd_mb_overview = pd.read_csv(path[:-len('/mbdata')] + '/mb_overview_seasonal_mb_time_periods_20220301.csv',
#                             index_col='Unnamed: 0')
#ref_candidates = pd_mb_overview.rgi_id.unique()

# for tests
#testing = testing
#if testing:
#    ref_candidates = ['RGI60-11.01450'] #rgis_w_mb_profiles #oggm.utils.get_ref_mb_glaciers_candidates()
#    working_dir = utils.gettempdir(dirname='OGGM_seasonal_mb_calib', reset=True)
#else:
#working_dir = '/home/lilianschuster/Schreibtisch/PhD/bayes_2022/calib_winter_mb/oggm_folder_from_cluster'
if cluster:
    working_dir = '/home/users/lschuster/Schuster_et_al_phd_paper_1_cluster/oggm_run_gdir_folder/node_folder'
else:
    working_dir = '/home/lilianschuster/Schreibtisch/PhD/Schuster_et_al_phd_paper_1/data/per_glacier_calib_data/oggm_folder_projections'

cfg.initialize()
cfg.PARAMS['use_multiprocessing'] = True #True
cfg.PARAMS['mp_processes'] = 28

cfg.PATHS['working_dir'] = working_dir
cfg.PARAMS['hydro_month_nh'] = 1
cfg.PARAMS['hydro_montpahh_sh'] = 1
cfg.PARAMS['continue_on_error'] = True
warnings.filterwarnings("ignore", category=DeprecationWarning) 

correction = False

#[ref_candidates[12]])
gdirs = workflow.init_glacier_directories(ref_candidates)
#gdirs= workflow.init_glacier_directories(rgis_w_mb_profiles)

2022-12-12 20:40:29: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2022-12-12 20:40:29: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2022-12-12 20:40:29: oggm.cfg: Multiprocessing: using all available processors (N=32)
2022-12-12 20:40:29: oggm.cfg: Multiprocessing switched ON after user settings.
2022-12-12 20:40:29: oggm.cfg: Multiprocessing: using the requested number of processors (N=28)
2022-12-12 20:40:29: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2022-12-12 20:40:29: oggm.workflow: Execute entity tasks [GlacierDirectory] on 247 glaciers


In [99]:
_doc = 'the calibrated melt_f, pf and temp.b to match geodetic mean observation, winter MB and approximately std of direct annual time series'
cfg.BASENAMES['calib_geod_opt_winter_mb_approx_std'] = ('calib_geod_opt_winter_mb_approx_std.json', _doc)

_doc2 = 'the calibrated melt_f and pf to match geodetic mean observation and winter MB bias, temp. bias set to zero!'
cfg.BASENAMES['calib_geod_opt_winter_mb_temp_b_0'] = ('calib_geod_opt_winter_mb_temp_b_0.json', _doc2)

_doc3 = 'the calibrated melt_f and pf to match geodetic mean observation and interannual direct glaciological standard deviation, temp. bias set to zero!'
cfg.BASENAMES['calib_geod_opt_std_temp_b_0'] = ('calib_geod_opt_std_temp_b_0.json', _doc3)

_doc4 = 'the calibrated melt_f to match geodetic mean observation, no temp. bias and globally cte prcp. fac extracted from median of ref glacier matching direct glaciological standard deviation'
cfg.BASENAMES['calib_only_geod_temp_b_0_pf_cte_via_std'] = ('calib_only_geod_temp_b_0_pf_cte_via_std.json', _doc4)
_doc5 = 'the calibrated melt_f to match geodetic mean observation, no temp. bias and per glacier one prcp. factor extracted from median of ref glacier matching direct glaciological standard deviation'
cfg.BASENAMES['calib_only_geod_temp_b_0_pf_fit_via_winter_mb'] = ('calib_only_geod_temp_b_0_pf_fit_via_winter_mb.json', _doc5)

ValueError: DocumentedDict accepts only tuple of len 2

In [100]:
from MBsandbox.help_func import get_mean_mb_profile_filtered
from MBsandbox.mbmod_daily_oneflowline import TIModel_Sfc_Type, TIModel

In [101]:
# for comparison, we just take those glaciers that work for all options, however!

# TODO: if this is wanted --> need to recompute the rgi_working_for_all !!!
rgi_working_for_all = pd.read_csv(f'/home/users/lschuster/Schuster_et_al_phd_paper_1_cluster/oggm_run_gdir_folder/rgi_working_for_all_list.csv',
                                  index_col = 'Unnamed: 0')
rgi_working_for_all = list(rgi_working_for_all['0'].values)
#pd_params_stats = pd_params_stats.loc[rgi_working_for_all]
pd_params_stats = pd.read_csv(f'/home/users/lschuster/Schuster_et_al_phd_paper_1_cluster/oggm_run_gdir_folder/pd_params_stats_working_for_all.csv', index_col = 0)
rgis_w_mb_profiles = pd_params_stats.mae_mb_profile.dropna().index.unique().values

pd_params_stats_mb_profile = pd_params_stats.loc[rgis_w_mb_profiles]

pd_params_stats_mb_profile

,rgi_id.1,temp_bias,pf_opt,melt_f,winter_prcp_mean,winter_solid_prcp_mean,specific_melt_winter_kg_m2,except_necessary,quot_std,mae_mb_profile,...,sfc_type_distinction,changing melt_f with sfc type,calib_type,daily_yearly_prcp_mean_historical,daily_winter_prcp_mean_historical,hemisphere,area,bias_winter_mb,mb_grad_type,sfc_type
rgi_id,,,,,,,,,,,,,,,,,,,,,
RGI60-02.00377,RGI60-02.00377,5.75,2.415065,81.989196,5126.116401,2761.575927,662.708431,0.0,0.914607,277.209136,...,True,linear,calib_geod_opt_winter_mb_approx_std,NaN,NaN,NaN,NaN,NaN,mb_monthly_cte,linear\nmonthly
RGI60-02.00377,RGI60-02.00377,4.75,2.311831,87.180710,4906.996648,2756.497468,645.761098,0.0,0.911616,212.775902,...,True,linear,calib_geod_opt_winter_mb_approx_std,NaN,NaN,NaN,NaN,NaN,mb_monthly_var_an_cycle,linear\nmonthly
RGI60-02.00377,RGI60-02.00377,5.75,2.549000,81.344870,5410.401513,2914.727916,818.116019,0.0,0.925840,296.600350,...,True,linear,calib_geod_opt_winter_mb_approx_std,NaN,NaN,NaN,NaN,NaN,mb_pseudo_daily_cte,linear\nmonthly
RGI60-02.00377,RGI60-02.00377,4.75,2.450724,86.464648,5201.805270,2922.105737,814.267905,0.0,0.924542,223.827498,...,True,linear,calib_geod_opt_winter_mb_approx_std,NaN,NaN,NaN,NaN,NaN,mb_pseudo_daily_var_an_cycle,linear\nmonthly
RGI60-02.00377,RGI60-02.00377,5.75,2.546546,81.243776,5405.191825,2911.921318,814.349588,0.0,0.935750,307.530267,...,True,linear,calib_geod_opt_winter_mb_approx_std,NaN,NaN,NaN,NaN,NaN,mb_pseudo_daily_fake_cte,linear\nmonthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RGI60-11.00804,RGI60-11.00804,0.00,3.300178,366.613891,NaN,NaN,NaN,0.0,1.633410,398.217764,...,True,neg_exp_annual_t1.0yr,calib_only_geod_temp_b_0_pf_fit_via_winter_mb,NaN,NaN,NaN,NaN,NaN,mb_pseudo_daily_fake_var_an_cycle,neg_exp\nannual_t1yr
RGI60-11.00804,RGI60-11.00804,0.00,3.300178,435.206806,NaN,NaN,NaN,0.0,1.709816,572.607749,...,True,linear_annual,calib_only_geod_temp_b_0_pf_fit_via_winter_mb,NaN,NaN,NaN,NaN,NaN,mb_real_daily_cte,linear\nannual
RGI60-11.00804,RGI60-11.00804,0.00,3.300178,430.983103,NaN,NaN,NaN,0.0,1.752904,548.921253,...,True,neg_exp_annual_t1.0yr,calib_only_geod_temp_b_0_pf_fit_via_winter_mb,NaN,NaN,NaN,NaN,NaN,mb_real_daily_cte,neg_exp\nannual_t1yr


In [102]:
import scipy
from scipy.stats import linregress

calib_types = pd_params_stats_mb_profile.calib_type.unique()
pd_stats_abl_grad = pd.DataFrame(np.NaN, index=np.arange(0, int(3*2*3*len(rgis_w_mb_profiles*5))), #*len(gdirs))),
                            columns=['rgi_id', 'temp_bias', 'pf', 'melt_f',
                                  'mod_ablation_gradient','obs_ablation_gradient',
                                     'mb_type', 'grad_type', 'melt_f_change', 'calib_type'])

t_melt = 0
j = 0
# gdir = gdirs[9] HEF
load = False
if load:
    for gdir in gdirs:
        try:
            print(gdir.rgi_id)
            try:
                mean_mb_prof = get_mean_mb_profile_filtered(gdir, input_fs='_monthly_W5E5', obs_ratio_needed=0.6)

            except:
                mean_mb_prof = get_mean_mb_profile_filtered(gdir, input_fs='_daily_W5E5', obs_ratio_needed=0.6)
            for calib_type in calib_types:
                for mb_type in ['mb_monthly', 'mb_pseudo_daily', 'mb_pseudo_daily_fake', 'mb_real_daily']:
                    for grad_type in ['cte', 'var_an_cycle']:
                        for melt_f_change_r in [False, 'linear', 'neg_exp_t1.0yr']:
                            if melt_f_change_r is False:
                                sfc_type_distinction = False
                                melt_f_change = 'False'
                            else:
                                sfc_type_distinction = True
                                melt_f_update = 'monthly'
                                if 'neg_exp' in melt_f_change_r:
                                    melt_f_change = 'neg_exp'
                                    if melt_f_change_r == 'neg_exp_t0.5yr':
                                        tau_e_fold_yr = 0.5
                                    elif melt_f_change_r == 'neg_exp_t1.0yr':
                                        tau_e_fold_yr = 1
                                else:
                                    melt_f_change = melt_f_change_r
                                    tau_e_fold_yr = np.NaN
                            if sfc_type_distinction:
                                gd_mb = TIModel_Sfc_Type(gdir, 200,
                                                         prcp_fac=1,
                                                         mb_type=mb_type,
                                                         grad_type=grad_type,
                                                         baseline_climate=climate_type,
                                                         melt_f_ratio_snow_to_ice=0.5, melt_f_update=melt_f_update,
                                                         melt_f_change=melt_f_change,
                                                         tau_e_fold_yr=tau_e_fold_yr,
                                                         t_melt=t_melt
                                                         )
                            else:
                                gd_mb = TIModel(gdir, 200,
                                                prcp_fac=1,
                                                mb_type=mb_type,
                                                grad_type=grad_type,
                                                baseline_climate=climate_type,
                                                t_melt=t_melt
                                                )


                            #if sfc_type_distinction is False:
                            #    sfc_type = False
                            #else:
                            sfc_type = melt_f_change

                            if (sfc_type is not False) and (sfc_type != 'False'):

                                fs_new = '_{}_sfc_type_{}_{}_{}'.format('W5E5', sfc_type, mb_type,
                                                                             grad_type)
                            else:
                                fs_new = '_{}_sfc_type_{}_{}_{}'.format('W5E5', sfc_type, mb_type, grad_type)
                            # json_filename = 'melt_f_geod_opt_winter_mb_approx_std'
                            # get the calibrated melt_f that suits to the prcp factor
                            try:
                                d = gdir.read_json(filename=calib_type,
                                                   filesuffix=fs_new)
                                # get the corrected ref_hgt so that we can apply this again on the mb model
                                # if otherwise not melt_f could be found!
                                precipitation_factor = d['pf']
                                melt_f_chosen = d['melt_f']
                                temperature_bias = d['temp_bias']
                            except:
                                raise InvalidWorkflowError(
                                    'there is no calibrated melt_f for this precipitation factor, glacier, climate'
                                    'mb_type and grad_type, need to do the calibration first!')

                            if precipitation_factor is not None:
                                gd_mb.prcp_fac = precipitation_factor

                            if temperature_bias is not None:
                                gd_mb.temp_bias = temperature_bias
                            gd_mb.melt_f = melt_f_chosen
                            hgts, widths = gdir.get_inversion_flowline_hw()
                            gd_mb.get_specific_mb(heights=hgts, widths=widths,
                                                      year=np.arange(1979, # yrs_seasonal_mbs[0]
                                                                     2019 + 1, 1))

                            obs_mean_mb_profile_filtered, obs_mean_mb_profile_years = mean_mb_prof

                            fac = cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density']
                            mb_annual = []
                            if sfc_type_distinction:
                                gd_mb.reset_pd_mb_bucket()
                            for y in obs_mean_mb_profile_years:
                                # if isinstance(gd_mb, TIModel):
                                mb_y = gd_mb.get_annual_mb(hgts, y) * fac
                                # print(h, mb_y)
                                # else:
                                #   mb_y = gd_mb.pd_mb_annual[y]
                                mb_annual.append(mb_y)
                            y_modelled = np.array(mb_annual).mean(axis=0)

                            h_condi = obs_mean_mb_profile_filtered.index
                            condi1 = ((hgts > min(h_condi)) & (hgts < max(h_condi)))
                            f = scipy.interpolate.interp1d(h_condi,
                                                           obs_mean_mb_profile_filtered.values,
                                                           kind='cubic')
                            y_modelled = y_modelled[condi1]
                            y_interp_obs = f(hgts[condi1])
                            try:
                                slope_mod, _,_,_,_ = linregress(hgts[condi1][y_modelled<0], y_modelled[y_modelled<0])
                                slope_obs, _,_,_,_ = linregress(hgts[condi1][y_interp_obs<0], y_interp_obs[y_interp_obs<0])
                            except:
                                slope_mod = np.NaN
                                slope_obs = np.NaN

                            pd_stats_abl_grad.loc[j, 'rgi_id'] = gdir.rgi_id
                            pd_stats_abl_grad.loc[j, 'temp_bias'] = temperature_bias
                            pd_stats_abl_grad.loc[j, 'pf'] = precipitation_factor
                            pd_stats_abl_grad.loc[j, 'melt_f'] = melt_f_chosen
                            pd_stats_abl_grad.loc[j, 'melt_f_change'] = melt_f_change
                            pd_stats_abl_grad.loc[j, 'mb_type'] = mb_type
                            pd_stats_abl_grad.loc[j, 'grad_type'] = grad_type
                            pd_stats_abl_grad.loc[j, 'calib_type'] = calib_type
                            pd_stats_abl_grad.loc[j, 'mod_ablation_gradient'] = slope_mod
                            pd_stats_abl_grad.loc[j, 'obs_ablation_gradient'] = slope_obs

                            j+=1
        except:
            print(gdir.rgi_id)

    pd_stats_abl_grad['quotient_mod_vs_obs_abl_grad'] = pd_stats_abl_grad['mod_ablation_gradient'] / pd_stats_abl_grad['obs_ablation_gradient']
    pd_stats_abl_grad = pd_stats_abl_grad.dropna()
    pd_stats_abl_grad['mb_grad_type'] =  pd_stats_abl_grad['mb_type'].values + '_'+ pd_stats_abl_grad['grad_type'].values
    pd_stats_abl_grad['sfc_type'] = pd_stats_abl_grad['melt_f_change'].values
    # pd_stats_abl_grad.loc[pd_stats_abl_grad.mb_type != 'mb_pseudo_daily_fake']
    pd_rgi_count = pd_stats_abl_grad.groupby('rgi_id').count()
    rgi_w_abl_grad = pd_rgi_count[(pd_rgi_count==120)].dropna() # 120
    for g in pd_stats_abl_grad.rgi_id.unique():
        if g not in rgi_w_abl_grad.index:
            print(g)
            pd_stats_abl_grad = pd_stats_abl_grad.loc[(pd_stats_abl_grad.rgi_id !=g)]
    pd_stats_abl_grad.to_csv(f'{fpath}/0_pd_stats_mb_ablation_grad.csv')
else:
    pd_stats_abl_grad = pd.read_csv(f'{fpath}/0_pd_stats_mb_ablation_grad.csv', index_col=0)
    pd_stats_abl_grad['difference_mod_vs_obs_abl_grad'] = pd_stats_abl_grad['mod_ablation_gradient'] - pd_stats_abl_grad['obs_ablation_gradient']


In [103]:
len(pd_stats_abl_grad.rgi_id.unique())

53

**now only look at calibration option e: need to redo the gradient computation because there are more glaciers that work (as they don't need to have e.g. winter MB...)**
- also it is different to the poster dataset as I added now again the pseudo_daily and 'neg_exp_t1.0yr'

In [109]:
#pd_params_stats = pd_params_stats.loc[rgi_working_for_all]
pd_params_stats = pd.read_csv(f'{fpath}/0_pd_params_stats_not_working_for_all.csv',
                                  index_col = 0)

ref_rgis_w_mb_profiles = pd_params_stats['mae_mb_profile'].dropna().index.unique()
gdirs_w_mb_profiles= workflow.init_glacier_directories(ref_rgis_w_mb_profiles)
from oggm.exceptions import InvalidWorkflowError
import scipy
from scipy.stats import linregress
print(len(ref_rgis_w_mb_profiles))

t_melt = 0
j = 0
load = False
if load:
    calib_types = pd_params_stats_mb_profile.calib_type.unique()
    pd_stats_mb_ablation_grad_ti_model_comparison = pd.DataFrame(np.NaN,
                                                                 index=np.arange(0, int(3*2*3*len(rgis_w_mb_profiles*5))), #*len(gdirs))),
                            columns=['rgi_id', 'temp_bias', 'pf', 'melt_f',
                                  'mod_ablation_gradient','obs_ablation_gradient',
                                     'mb_type', 'grad_type', 'melt_f_change', 'calib_type'])
    for gdir in gdirs_w_mb_profiles:
        try:
            mean_mb_prof = get_mean_mb_profile_filtered(gdir, input_fs='_monthly_W5E5', obs_ratio_needed=0.6)

        except:
            mean_mb_prof = get_mean_mb_profile_filtered(gdir, input_fs='_daily_W5E5', obs_ratio_needed=0.6)
        try:
            for calib_type in [calib_types[-1]]: # only look at calib option e!!!
                for mb_type in ['mb_monthly', 'mb_pseudo_daily',
                                'mb_pseudo_daily_fake','mb_real_daily']:
                    for grad_type in ['cte', 'var_an_cycle']:
                        for melt_f_change_r in [False, 'neg_exp_t1.0yr', 'linear']:
                            if melt_f_change_r is False:
                                sfc_type_distinction = False
                                melt_f_change = 'False'
                            else:
                                sfc_type_distinction = True
                                melt_f_update = 'monthly'
                                if 'neg_exp' in melt_f_change_r:
                                    melt_f_change = 'neg_exp'
                                    if melt_f_change_r == 'neg_exp_t0.5yr':
                                        tau_e_fold_yr = 0.5
                                    elif melt_f_change_r == 'neg_exp_t1.0yr':
                                        tau_e_fold_yr = 1
                                else:
                                    melt_f_change = melt_f_change_r
                                    tau_e_fold_yr = np.NaN
                            if sfc_type_distinction:
                                gd_mb = TIModel_Sfc_Type(gdir, 200,
                                                         prcp_fac=1,
                                                         mb_type=mb_type,
                                                         grad_type=grad_type,
                                                         baseline_climate=climate_type,
                                                         melt_f_ratio_snow_to_ice=0.5, melt_f_update=melt_f_update,
                                                         melt_f_change=melt_f_change,
                                                         tau_e_fold_yr=tau_e_fold_yr,
                                                         t_melt=t_melt
                                                         )
                            else:
                                gd_mb = TIModel(gdir, 200,
                                                prcp_fac=1,
                                                mb_type=mb_type,
                                                grad_type=grad_type,
                                                baseline_climate=climate_type,
                                                t_melt=t_melt
                                                )


                            #if sfc_type_distinction is False:
                            #    sfc_type = False
                            #else:
                            sfc_type = melt_f_change

                            if (sfc_type is not False) and (sfc_type != 'False'):

                                fs_new = '_{}_sfc_type_{}_{}_{}'.format('W5E5', sfc_type, mb_type,
                                                                             grad_type)
                            else:
                                fs_new = '_{}_sfc_type_{}_{}_{}'.format('W5E5', sfc_type, mb_type, grad_type)
                            # json_filename = 'melt_f_geod_opt_winter_mb_approx_std'
                            # get the calibrated melt_f that suits to the prcp factor
                            try:
                                d = gdir.read_json(filename=calib_type,
                                                   filesuffix=fs_new)
                                # get the corrected ref_hgt so that we can apply this again on the mb model
                                # if otherwise not melt_f could be found!
                                precipitation_factor = d['pf']
                                melt_f_chosen = d['melt_f']
                                temperature_bias = d['temp_bias']
                            except:
                                raise InvalidWorkflowError(
                                    'there is no calibrated melt_f for this precipitation factor, glacier, climate'
                                    'mb_type and grad_type, need to do the calibration first!')

                            if precipitation_factor is not None:
                                gd_mb.prcp_fac = precipitation_factor

                            if temperature_bias is not None:
                                gd_mb.temp_bias = temperature_bias
                            gd_mb.melt_f = melt_f_chosen
                            hgts, widths = gdir.get_inversion_flowline_hw()
                            gd_mb.get_specific_mb(heights=hgts, widths=widths,
                                                      year=np.arange(1979, # yrs_seasonal_mbs[0]
                                                                     2019 + 1, 1))

                            obs_mean_mb_profile_filtered, obs_mean_mb_profile_years = mean_mb_prof

                            fac = cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density']
                            mb_annual = []
                            if sfc_type_distinction:
                                gd_mb.reset_pd_mb_bucket()
                            for y in obs_mean_mb_profile_years:
                                # if isinstance(gd_mb, TIModel):
                                mb_y = gd_mb.get_annual_mb(hgts, y) * fac
                                # print(h, mb_y)
                                # else:
                                #   mb_y = gd_mb.pd_mb_annual[y]
                                mb_annual.append(mb_y)
                            y_modelled = np.array(mb_annual).mean(axis=0)

                            h_condi = obs_mean_mb_profile_filtered.index
                            condi1 = ((hgts > min(h_condi)) & (hgts < max(h_condi)))
                            f = scipy.interpolate.interp1d(h_condi,
                                                           obs_mean_mb_profile_filtered.values,
                                                           kind='cubic')
                            y_modelled = y_modelled[condi1]
                            y_interp_obs = f(hgts[condi1])
                            try:
                                slope_mod, _,_,_,_ = linregress(hgts[condi1][y_modelled<0], y_modelled[y_modelled<0])
                                slope_obs, _,_,_,_ = linregress(hgts[condi1][y_interp_obs<0], y_interp_obs[y_interp_obs<0])
                            except:
                                slope_mod = np.NaN
                                slope_obs = np.NaN

                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'rgi_id'] = gdir.rgi_id
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'temp_bias'] = temperature_bias
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'pf'] = precipitation_factor
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'melt_f'] = melt_f_chosen
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'melt_f_change'] = melt_f_change
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'mb_type'] = mb_type
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'grad_type'] = grad_type
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'calib_type'] = calib_type
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'mod_ablation_gradient'] = slope_mod
                            pd_stats_mb_ablation_grad_ti_model_comparison.loc[j, 'obs_ablation_gradient'] = slope_obs

                            j+=1
        except:
            print(gdir.rgi_id)

    pd_stats_mb_ablation_grad_ti_model_comparison['quotient_mod_vs_obs_abl_grad'] = pd_stats_mb_ablation_grad_ti_model_comparison['mod_ablation_gradient'] / pd_stats_mb_ablation_grad_ti_model_comparison['obs_ablation_gradient']
    pd_stats_mb_ablation_grad_ti_model_comparison = pd_stats_mb_ablation_grad_ti_model_comparison.dropna()
    pd_stats_mb_ablation_grad_ti_model_comparison['mb_grad_type'] =  pd_stats_mb_ablation_grad_ti_model_comparison['mb_type'].values + '_'+ pd_stats_mb_ablation_grad_ti_model_comparison['grad_type'].values
    pd_stats_mb_ablation_grad_ti_model_comparison['sfc_type'] = pd_stats_mb_ablation_grad_ti_model_comparison['melt_f_change'].values
    pd_stats_mb_ablation_grad_ti_model_comparison['difference_mod_vs_obs_abl_grad'] = pd_stats_mb_ablation_grad_ti_model_comparison['mod_ablation_gradient'] - pd_stats_mb_ablation_grad_ti_model_comparison['obs_ablation_gradient']
    
    pd_rgi_count = pd_stats_mb_ablation_grad_ti_model_comparison.groupby('rgi_id').count()
    rgi_w_abl_grad = pd_rgi_count[(pd_rgi_count==24)].dropna() # 120
    for g in pd_stats_mb_ablation_grad_ti_model_comparison.rgi_id.unique():
        if g not in rgi_w_abl_grad.index:
            print(g)
            pd_stats_mb_ablation_grad_ti_model_comparison = pd_stats_mb_ablation_grad_ti_model_comparison.loc[(pd_stats_mb_ablation_grad_ti_model_comparison.rgi_id !=g)]
    pd_stats_mb_ablation_grad_ti_model_comparison.to_csv(f'{fpath}/0_pd_stats_mb_ablation_grad_ti_model_comparison.csv')
    
else:
    pd_stats_mb_ablation_grad_ti_model_comparison = pd.read_csv(f'{fpath}/0_pd_stats_mb_ablation_grad_ti_model_comparison.csv',
                                                                index_col=0)


2022-12-12 20:42:17: oggm.workflow: Execute entity tasks [GlacierDirectory] on 93 glaciers


93


In [112]:
len(pd_stats_mb_ablation_grad_ti_model_comparison.rgi_id.unique())

80

In [96]:
pd_stats_mb_ablation_grad_ti_model_comparison.mb_type.unique()

array(['mb_monthly', 'mb_pseudo_daily', 'mb_real_daily'], dtype=object)

**median observed MB ablation gradient for 80 glaciers of C5**

In [113]:
np.quantile(pd_stats_mb_ablation_grad_ti_model_comparison.groupby(by='rgi_id').mean()['obs_ablation_gradient'],
            q=[0.025, 0.25,0.5,0.75,0.975])

array([ 1.98501604,  4.71559303,  6.54702968,  8.83013265, 18.39355234])

**median modelled MB ablation gradient for 80 glaciers of C5**

In [114]:
np.quantile(pd_stats_mb_ablation_grad_ti_model_comparison['mod_ablation_gradient'],
            q=[0.025, 0.25,0.5,0.75,0.975])

array([ 2.10877899,  4.41846078,  6.74087705,  9.4558949 , 16.58695314])